In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

In [2]:
#file paths

output_dirs = ['/ihme/costeffectiveness/results/vivarium_gates_bep/lbwsg_acmr/india/2020_02_24_01_05_37/count_data/',
               '/ihme/costeffectiveness/results/vivarium_gates_bep/lbwsg_acmr/mali/2020_02_24_01_07_25/count_data/',
               '/ihme/costeffectiveness/results/vivarium_gates_bep/lbwsg_acmr/pakistan/2020_02_24_01_08_41/count_data/',
               '/ihme/costeffectiveness/results/vivarium_gates_bep/lbwsg_acmr/tanzania/2020_02_24_01_10_34/count_data/']

locations = ['India','Mali','Pakistan','Tanzania']

In [19]:
master_avg = pd.DataFrame()
master_draws = pd.DataFrame()
master_counts = pd.DataFrame()

for i in list(range(0,4)):
     
    ylls = (pd.read_hdf(output_dirs[i] + 'ylls.hdf')
          .groupby(['input_draw','scenario']).sum().reset_index().rename(columns={'value':'ylls'}))
    ylds = (pd.read_hdf(output_dirs[i] + 'ylds.hdf')
          .groupby(['input_draw','scenario']).sum().reset_index().rename(columns={'value':'ylds'}))
    dalys = ylls.merge(ylds, right_on=['input_draw','scenario'], left_on=['input_draw','scenario'])
    dalys['dalys'] = dalys['ylls'] + dalys['ylds']
    dalys = dalys.drop(columns=['ylls','ylds'])
   
    treatment = pd.read_hdf(output_dirs[i] + 'population.hdf').reset_index().groupby(['input_draw','scenario','treatment_group','measure']).sum().reset_index().drop(columns='index').rename(columns={'value':'population'})
    treatment = treatment.where(treatment['measure'] == 'total_population').dropna()
    treatment = pd.pivot_table(treatment, index=['input_draw','scenario'], columns='treatment_group', values='population').reset_index()
    treatment = treatment.rename(columns={'all':'total_population',
                               'bep':'bep_population',
                               'ifa':'ifa_population',
                               'mmn':'mmn_population',
                               'none':'untreated_population'})
    
    dalys_tot = dalys.merge(treatment, right_on=['input_draw','scenario'], 
                          left_on=['input_draw','scenario'])
    dalys_tot.head()   

    count = dalys_tot
    count['location'] = locations[i]
    master_counts = master_counts.append(count)

    # add cost data
    #baseline dalys and cost
    dalys_baseline = pd.DataFrame.copy(dalys_tot.where(dalys_tot['scenario'] == 'baseline').dropna().drop(columns='scenario'))
    dalys_baseline['cost'] = 2.69 * dalys_baseline['ifa_population'] 
    
    #mmn dalys and cost
    dalys_mmn = pd.DataFrame.copy(dalys_tot.where(dalys_tot['scenario'] == 'mmn_scale_up').dropna().drop(columns='scenario'))
    dalys_mmn['cost'] = 3.69 * dalys_mmn['mmn_population'] 
  
    #BEP universal dalys and universal cost
    
    #determine variables
    icer_scenario= "icer_bep_targeted_rel_to_baseline"
    #icer_bep_universal_rel_to_baseline: 25, 50, 200
    #icer_bep_targeted_rel_to_baseline
    #icer_bep_universal_rel_to_mmn
    #icer_bep_targeted_rel_to_mmn
    
    cost_target = 25
        
    #initialize bep cost and icer
    bep_cost = -20
    icer=0
       
    while  icer < cost_target:
        
        #BEP dalys and cost
        dalys_bep_universal = (pd.DataFrame
                      .copy(dalys_tot.where(dalys_tot['scenario'] == 'bep_scale_up')
                      .dropna().drop(columns='scenario')))
        dalys_bep_universal['cost'] = bep_cost * dalys_bep_universal['bep_population'] 
     
        dalys_bep_targeted = (pd.DataFrame
                      .copy(dalys_tot.where(dalys_tot['scenario'] == 'bep_targeted_scale_up')
                      .dropna().drop(columns='scenario')))
        dalys_bep_targeted['cost'] = (bep_cost * dalys_bep_targeted['bep_population'] + 3.69 * dalys_bep_targeted['mmn_population'] )

        dalys_scenarios1 = dalys_baseline.merge(dalys_mmn, right_on=['input_draw'],
                                 left_on=['input_draw'], suffixes=['_baseline','_mmn'])
        dalys_scenarios2 = dalys_baseline.merge(dalys_bep_universal, right_on=['input_draw'],
                                 left_on=['input_draw'], suffixes=['_baseline','_bep'])
        dalys_scenarios3 = dalys_baseline.merge(dalys_bep_targeted, right_on=['input_draw'],
                                 left_on=['input_draw'], suffixes=['_baseline','_bep_targeted'])
        dalys_scenarios4 = dalys_scenarios1.merge(dalys_scenarios2, right_on=['input_draw'],
                                 left_on=['input_draw'], suffixes=['','_drop'])
        dalys_scenarios = dalys_scenarios4.merge(dalys_scenarios3, right_on=['input_draw'],
                                 left_on=['input_draw'], suffixes=['','_drop'])
        dalys_scenarios = dalys_scenarios.drop(columns=[c for c in dalys_scenarios.columns if 'drop' in c])

        #ICERS
        #mmn vs baseline
        dalys_scenarios['cost_averted_mmn'] = dalys_scenarios['cost_mmn'] - dalys_scenarios['cost_baseline']
        dalys_scenarios['dalys_averted_mmn'] = dalys_scenarios['dalys_baseline'] - dalys_scenarios['dalys_mmn']
        dalys_scenarios['icer_mmn_rel_to_baseline'] = dalys_scenarios['cost_averted_mmn'] / dalys_scenarios['dalys_averted_mmn']
        
        #bep universal vs baseline
        dalys_scenarios['cost_averted_bep_universal'] = dalys_scenarios['cost_bep'] - dalys_scenarios['cost_baseline']
        dalys_scenarios['dalys_averted_bep_universal'] = dalys_scenarios['dalys_baseline'] - dalys_scenarios['dalys_bep']
        dalys_scenarios['icer_bep_universal_rel_to_baseline'] = dalys_scenarios['cost_averted_bep_universal'] / dalys_scenarios['dalys_averted_bep_universal']
        
        #bep targeted vs baseline
        dalys_scenarios['cost_averted_bep_targeted'] = dalys_scenarios['cost_bep_targeted'] - dalys_scenarios['cost_baseline']
        dalys_scenarios['dalys_averted_bep_targeted'] = dalys_scenarios['dalys_baseline'] - dalys_scenarios['dalys_bep_targeted']
        dalys_scenarios['icer_bep_targeted_rel_to_baseline'] = dalys_scenarios['cost_averted_bep_targeted'] / dalys_scenarios['dalys_averted_bep_targeted']
            
        #bep universal vs. mmn
        dalys_scenarios['cost_averted_bep_rel_to_mmn'] = dalys_scenarios['cost_bep'] - dalys_scenarios['cost_mmn']
        dalys_scenarios['dalys_averted_bep_rel_to_mmn'] = dalys_scenarios['dalys_mmn'] - dalys_scenarios['dalys_bep']
        dalys_scenarios['icer_bep_universal_rel_to_mmn'] = dalys_scenarios['cost_averted_bep_rel_to_mmn'] / dalys_scenarios['dalys_averted_bep_rel_to_mmn']


        #bep targeted vs mmn
        dalys_scenarios['cost_averted_bep_targeted_rel_to_mmn'] = dalys_scenarios['cost_bep_targeted'] - dalys_scenarios['cost_mmn']
        dalys_scenarios['dalys_averted_bep_targeted_rel_to_mmn'] = dalys_scenarios['dalys_mmn'] - dalys_scenarios['dalys_bep_targeted']
        dalys_scenarios['icer_bep_targeted_rel_to_mmn'] = dalys_scenarios['cost_averted_bep_targeted_rel_to_mmn'] / dalys_scenarios['dalys_averted_bep_targeted_rel_to_mmn']

        dalys_scenarios = dalys_scenarios.set_index(['input_draw'])
        dalys_scenarios['location'] = locations[i]
        
        dalys_prep = dalys_scenarios.reset_index().drop(columns=['input_draw'])
        dalys_mean = pd.DataFrame(dalys_prep.mean())
        dalys_min = pd.DataFrame(dalys_prep.quantile(0.025))
        dalys_max = pd.DataFrame(dalys_prep.quantile(0.975))
        per_pt1 = dalys_mean.merge(dalys_min, right_index=True, left_index=True)
        
        final_per_pt = per_pt1.merge(dalys_max, right_index=True, left_index=True)#, suffixes=['','_drop'])
        final_per_pt = final_per_pt.rename(columns={0.0:'mean', 0.025:'min', 0.975:'max'})#.drop(columns='0.0_mean_drop')
        final_per_pt = final_per_pt.reset_index()
        final_per_pt['location'] = locations[i]
        
        #Printing ICER
        icer = final_per_pt.loc[final_per_pt['index']==icer_scenario,'mean'].iloc[0]
                
        if icer < cost_target:
            bep_cost=bep_cost + 0.10
        
    print(locations[i])
    print('bep cost=', round(bep_cost-0.1,1))
    print(icer_scenario,'=', icer)
            
    master_draws = master_draws.append(dalys_scenarios.reset_index())
    master_avg = master_avg.append(final_per_pt)

India
bep cost= 10.5
icer_bep_targeted_rel_to_baseline = 25.005064742905052
Mali
bep cost= 14.5
icer_bep_targeted_rel_to_baseline = 25.012516136032477
Pakistan
bep cost= 15.1
icer_bep_targeted_rel_to_baseline = 25.010679708722506
Tanzania
bep cost= -10.0
icer_bep_targeted_rel_to_baseline = 25.066580017437456
